In [1]:
import glob
import os
import json
import nibabel as nib
from skimage.metrics import structural_similarity as ssim
import numpy as np
import tensorflow as tf
from skimage.exposure import rescale_intensity
import pandas as pd
import datetime

In [ ]:
# !unzip "/content/drive/MyDrive/GP/Zip Synth Datasets/T2-FLAIR/ResNet-7(30 ep) - FLAIR Synth from T2 - BraTS Validation - Brain Cropped.zip"
# !unzip "/content/drive/MyDrive/GP/Zip Synth Datasets/T2-FLAIR/ResNet-7(30 ep) - T2 Synth from FLAIR - BraTS Validation - Brain Cropped.zip"

# !unzip "/content/drive/MyDrive/GP/FLAIR BraTS Validation Set.zip"
# !unzip '/content/drive/MyDrive/GP/T1c Validation.zip' -d '/content/Synthesized BraTS Validation Brain Cropped'

!unzip '/content/drive/MyDrive/GP/Synthesized BraTS Validation Brain Cropped.zip'

# FLAIR Beginning FROM GANS NOT CYCLEGAN

In [9]:
root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'
# root_path = '/content/FLAIR BraTS Validation Set'

# T1_root_path = '/content/drive/MyDrive/GP/FLAIR GANs Fusion/55 epochs FLAIR/50-50 T2+T1c'
T1_root_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T1 - 29 July 2023'

data_list = sorted(glob.glob(root_path + '/*'))
T1_data_list = sorted(glob.glob(T1_root_path + '/*'))

T1_T2_SSIM = []
True_T1_paths = []
Synth_T1_paths = []


# Read the contents of the file
with open('/content/drive/MyDrive/GP/validation.txt', 'r') as file:
    contents = file.read()

# Replace single quotes with double quotes
contents = contents.replace("'", "\"")

# Load the JSON array
validation_names = json.loads(contents)

In [10]:
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            True_T1_paths.append(file_path)

for dirpath, dirnames, filenames in os.walk(T1_root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            Synth_T1_paths.append(file_path)

True_T1_paths = sorted(True_T1_paths)
Synth_T1_paths = sorted(Synth_T1_paths)

In [5]:
print(len(True_T1_paths), len(Synth_T1_paths), len(data_list), len(T1_data_list))

251 251 251 251


In [11]:
# calculate the SSIM score for each slice of each pair of volumes
def calc_2D_tf_SSIM(Synth_paths, True_paths):
    SSIM_tf_2D = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        # calculate the SSIM score for each slice
        ssim_scores_vol = []

        for i in range(vol1.shape[2]):
            slice1 = vol1[:, :, i]
            slice2 = vol2[:, :, i]

            slice1 = np.stack([slice1]*3, axis=-1)
            slice2 = np.stack([slice2]*3, axis=-1)

            ssim_score = tf.image.ssim(slice1, slice2, max_val=1)
            ssim_scores_vol.append(ssim_score)


        # calculate the average SSIM score for the volume
        avg_ssim_vol = np.mean(ssim_scores_vol)

        # add the average SSIM scores to the list
        SSIM_tf_2D.append(avg_ssim_vol)

    return SSIM_tf_2D

def _calculate_mae_mse(arr1, arr2):
    diff = arr1 - arr2
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff ** 2)
    return mae, mse

def calculate_mae_mse(T1_Synth_paths, T1_True_paths, norm_range):
    MAE, MSE = [], []
    for path1, path2 in zip(T1_Synth_paths, T1_True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()

        vol1 = rescale_intensity(vol1, out_range=norm_range)
        vol2 = rescale_intensity(vol2, out_range=norm_range)

        mae, mse = _calculate_mae_mse(vol1, vol2)

        MAE.append(mae)
        MSE.append(mse)
    return MAE, MSE

def psnr(Synth_paths, True_paths):
    """
    Calculate the average PSNR between corresponding slices of two 3D volumes.
    """
    PSNR = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        psnrs = []
        for i in range(vol1.shape[2]):
            mse = np.mean((vol1[:,:,i] - vol2[:,:,i]) ** 2)
            if mse == 0:
                psnrs.append(float('inf'))
            else:
                max_pixel = np.max(vol1[:,:,i])
                psnrs.append(20 * np.log10(max_pixel / np.sqrt(mse)))
        avg_psnr = np.mean(psnrs)
        PSNR.append(avg_psnr)
    return PSNR

In [12]:
a = datetime.datetime.now()
SSIM_2D_T1_Synth_from_T2 = calc_2D_tf_SSIM(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

a = datetime.datetime.now()
MAE_T1_Synth_from_T2_0_1, MSE_T1_Synth_from_T2_0_1 = calculate_mae_mse(Synth_T1_paths, True_T1_paths, norm_range=(0, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

a = datetime.datetime.now()
PSNR_T1_Synth_from_T2 = psnr(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

19.519740533333334
1.2747735166666667
1.25417145


In [13]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    "Validation Set": validation_names,
    'SSIM - FLAIR Synth from T1': SSIM_2D_T1_Synth_from_T2,
    'MAE - FLAIR Synth from T1': MAE_T1_Synth_from_T2_0_1,
    'MSE - FLAIR Synth from T1': MSE_T1_Synth_from_T2_0_1,
    'PSNR - FLAIR Synth from T1': PSNR_T1_Synth_from_T2,
})

#/content/drive/MyDrive/GP/GANS - FLAIR From T1 (NO SSIM) (25) - 1 June
# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/GP/FINAL MODELS/FINAL METRICS/FINAL-GANS-FLAIR From T1 - 29 July 2023.csv', index=False)

In [14]:
root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'
T1_root_path = '/content/drive/MyDrive/GP/FINAL MODELS/Predicted Data/FINAL-GANS-FLAIR From T2 - 29 July 2023'
data_list = sorted(glob.glob(root_path + '/*'))
T1_data_list = sorted(glob.glob(T1_root_path + '/*'))
T1_T2_SSIM = []
True_T1_paths = []
Synth_T1_paths = []
# Read the contents of the file
with open('/content/drive/MyDrive/GP/validation.txt', 'r') as file:
    contents = file.read()
# Replace single quotes with double quotes
contents = contents.replace("'", "\"")
# Load the JSON array
validation_names = json.loads(contents)
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            True_T1_paths.append(file_path)

for dirpath, dirnames, filenames in os.walk(T1_root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            Synth_T1_paths.append(file_path)
True_T1_paths = sorted(True_T1_paths)
Synth_T1_paths = sorted(Synth_T1_paths)
print(len(True_T1_paths), len(Synth_T1_paths), len(data_list), len(T1_data_list))
a = datetime.datetime.now()
SSIM_2D_T1_Synth_from_T2 = calc_2D_tf_SSIM(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

a = datetime.datetime.now()
MAE_T1_Synth_from_T2_0_1, MSE_T1_Synth_from_T2_0_1 = calculate_mae_mse(Synth_T1_paths, True_T1_paths, norm_range=(0, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

a = datetime.datetime.now()
PSNR_T1_Synth_from_T2 = psnr(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)
# Create a DataFrame from the lists
df = pd.DataFrame({
    "Validation Set": validation_names,
    'SSIM - FLAIR Synth from T1': SSIM_2D_T1_Synth_from_T2,
    'MAE - FLAIR Synth from T1': MAE_T1_Synth_from_T2_0_1,
    'MSE - FLAIR Synth from T1': MSE_T1_Synth_from_T2_0_1,
    'PSNR - FLAIR Synth from T1': PSNR_T1_Synth_from_T2,
})

#/content/drive/MyDrive/GP/GANS - FLAIR From T1 (NO SSIM) (25) - 1 June
# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/GP/FINAL MODELS/FINAL METRICS/FINAL-GANS-FLAIR From T2 - 29 July 2023.csv', index=False)

251 251 251 251
17.138935183333334
1.2799104
1.2630753666666665


# FLAIR Ending FROM GANS NOT CYCLEGAN

In [ ]:
#T1->FLAIR
#T2->T2
T1_root_path = 'ResNet-7(30 ep) - FLAIR Synth from T2 - BraTS Validation - Brain Cropped'
T2_root_path = 'ResNet-7(30 ep) - T2 Synth from FLAIR - BraTS Validation - Brain Cropped'
root_path = '/content/drive/MyDrive/GP/BraTS Validation Brain Cropped'


data_list = sorted(glob.glob(root_path + '/*'))
T1_data_list = sorted(glob.glob(T1_root_path + '/*'))
T2_data_list = sorted(glob.glob(T2_root_path + '/*'))

T1_T2_SSIM = []
T2_T1_SSIM = []

True_T1_paths = []
True_T2_paths = []
Synth_T1_paths = []
Synth_T2_paths = []

# import json

# Read the contents of the file
with open('/content/drive/MyDrive/GP/validation.txt', 'r') as file:
    contents = file.read()

# Replace single quotes with double quotes
contents = contents.replace("'", "\"")

# Load the JSON array
validation_names = json.loads(contents)

In [ ]:
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            True_T1_paths.append(file_path)

for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('t2.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            True_T2_paths.append(file_path)

for dirpath, dirnames, filenames in os.walk(T1_root_path):
    for filename in filenames:
        if filename.endswith('flair.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            Synth_T1_paths.append(file_path)

for dirpath, dirnames, filenames in os.walk(T2_root_path):
    for filename in filenames:
        if filename.endswith('t2.nii.gz'):
            file_path = os.path.join(dirpath, filename)
            Synth_T2_paths.append(file_path)

In [ ]:
True_T1_paths, True_T2_paths = sorted(True_T1_paths), sorted(True_T2_paths)
Synth_T1_paths, Synth_T2_paths = sorted(Synth_T1_paths), sorted(Synth_T2_paths)

In [ ]:
print(len(True_T1_paths), len(True_T2_paths), len(Synth_T1_paths), len(Synth_T2_paths), len(data_list), len(T1_data_list), len(T2_data_list))

251 251 110 251 251 110 251


In [ ]:
# calculate the SSIM score for each slice of each pair of volumes
def calc_2D_tf_SSIM(Synth_paths, True_paths):
    SSIM_tf_2D = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        # calculate the SSIM score for each slice
        ssim_scores_vol = []

        for i in range(vol1.shape[2]):
            slice1 = vol1[:, :, i]
            slice2 = vol2[:, :, i]

            slice1 = np.stack([slice1]*3, axis=-1)
            slice2 = np.stack([slice2]*3, axis=-1)

            ssim_score = tf.image.ssim(slice1, slice2, max_val=1)
            ssim_scores_vol.append(ssim_score)


        # calculate the average SSIM score for the volume
        avg_ssim_vol = np.mean(ssim_scores_vol)

        # add the average SSIM scores to the list
        SSIM_tf_2D.append(avg_ssim_vol)

    return SSIM_tf_2D

def _calculate_mae_mse(arr1, arr2):
    diff = arr1 - arr2
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff ** 2)
    return mae, mse

def calculate_mae_mse(T1_Synth_paths, T1_True_paths, norm_range):
    MAE, MSE = [], []
    for path1, path2 in zip(T1_Synth_paths, T1_True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()

        vol1 = rescale_intensity(vol1, out_range=norm_range)
        vol2 = rescale_intensity(vol2, out_range=norm_range)

        mae, mse = _calculate_mae_mse(vol1, vol2)

        MAE.append(mae)
        MSE.append(mse)
    return MAE, MSE

def psnr(Synth_paths, True_paths):
    """
    Calculate the average PSNR between corresponding slices of two 3D volumes.
    """
    PSNR = []
    for path1, path2 in zip(Synth_paths, True_paths):
        # load the volumes from files
        vol1 = nib.load(path1).get_fdata()
        vol2 = nib.load(path2).get_fdata()
        vol1 = rescale_intensity(vol1, out_range=(0, 1))
        vol2 = rescale_intensity(vol2, out_range=(0, 1))

        psnrs = []
        for i in range(vol1.shape[2]):
            mse = np.mean((vol1[:,:,i] - vol2[:,:,i]) ** 2)
            if mse == 0:
                psnrs.append(float('inf'))
            else:
                max_pixel = np.max(vol1[:,:,i])
                psnrs.append(20 * np.log10(max_pixel / np.sqrt(mse)))
        avg_psnr = np.mean(psnrs)
        PSNR.append(avg_psnr)
    return PSNR

In [ ]:
a = datetime.datetime.now()
SSIM_2D_T1_Synth_from_T2 = calc_2D_tf_SSIM(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
SSIM_2D_T2_Synth_from_T1 = calc_2D_tf_SSIM(Synth_T2_paths, True_T2_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
MAE_T1_Synth_from_T2_0_1, MSE_T1_Synth_from_T2_0_1 = calculate_mae_mse(Synth_T1_paths, True_T1_paths, norm_range=(0, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
MAE_T2_Synth_from_T1_0_1, MSE_T2_Synth_from_T1_0_1 = calculate_mae_mse(Synth_T2_paths, True_T2_paths, norm_range=(0, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
MAE_T1_Synth_from_T2_neg1_1, MSE_T1_Synth_from_T2_neg1_1 = calculate_mae_mse(Synth_T1_paths, True_T1_paths, norm_range=(-1, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
MAE_T2_Synth_from_T1_neg1_1, MSE_T2_Synth_from_T1_neg1_1 = calculate_mae_mse(Synth_T2_paths, True_T2_paths, norm_range=(-1, 1))
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

a = datetime.datetime.now()
PSNR_T1_Synth_from_T2 = psnr(Synth_T1_paths, True_T1_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)


a = datetime.datetime.now()
PSNR_T2_Synth_from_T1 = psnr(Synth_T2_paths, True_T2_paths)
b = datetime.datetime.now()
delta = b - a
print(delta.total_seconds()/60)

KeyboardInterrupt: ignored

In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    "Validation Set": validation_names,
    'SSIM - T1 Synth from FLAIR': SSIM_2D_T1_Synth_from_T2,
    'SSIM - FLAIR Synth from T1': SSIM_2D_T2_Synth_from_T1,
    'MAE - T1 Synth from FLAIR': MAE_T1_Synth_from_T2_0_1,
    'MAE - FLAIR Synth from T1': MAE_T2_Synth_from_T1_0_1,
    'MSE - T1 Synth from FLAIR': MSE_T1_Synth_from_T2_0_1,
    'MSE - FLAIR Synth from T1': MSE_T2_Synth_from_T1_0_1,
    'PSNR - T1 Synth from FLAIR': PSNR_T1_Synth_from_T2,
    'PSNR - FLAIR Synth from T1': PSNR_T2_Synth_from_T1
})

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/GP/Metrics/SqzAtt(30 ep) - T1-FLAIR.csv', index=False)

In [ ]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    "Validation Set": validation_names,
    'SSIM - T1 Synth from T2': SSIM_2D_T1_Synth_from_T2,
    'SSIM - T2 Synth from T1': SSIM_2D_T2_Synth_from_T1,
    'MAE - T1 Synth from T2': MAE_T1_Synth_from_T2_0_1,
    'MAE - T2 Synth from T1': MAE_T2_Synth_from_T1_0_1,
    'MSE - T1 Synth from T2': MSE_T1_Synth_from_T2_0_1,
    'MSE - T2 Synth from T1': MSE_T2_Synth_from_T1_0_1,
    'PSNR - T1 Synth from T2': PSNR_T1_Synth_from_T2,
    'PSNR - T2 Synth from T1': PSNR_T2_Synth_from_T1
})

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/GP/Metrics/SqzAtt(30 ep) - T1-T2.csv', index=False)